# Prepare Data.

In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import re
import pytz
# import deepcut
from datetime import datetime, timezone
from pythainlp import word_tokenize
from pythainlp.corpus.common import thai_stopwords

# Read data.

In [2]:
def read_data(tweets_data_path):
    """ Function read data. """
    tweets_data = []
    tweets_file = open(tweets_data_path, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            tweets_data.append(tweet)
        except:
            continue
    tweets_file.close()
    return tweets_data

In [ ]:
# Combine data into a list tweets_data.
# tweets_data = read_data('data/28-09-62.txt')
# tweets_data = read_data('data/28-09-62-2.txt')
tweets_data = read_data('data/28-09-62-3.txt')
# tweets_data = read_data('data/29-09-62.txt')
# tweets_data = read_data('data/29-09-62-2.txt')

In [ ]:
# tweets_data[:10]

In [ ]:
len(tweets_data)

## Reformat time and convert time zones UTC To ICT

In [ ]:
thai_tz = pytz.timezone('Asia/Bangkok') # thai time zone

In [ ]:
# find error in data
lst_error = []
for i in range(len(tweets_data)):
    try:
        tweets_data[i]['created_at'].split(' ')
    except:
        lst_error.append(i)

In [ ]:
print(lst_error)
print('----------')
print(len(lst_error))

In [ ]:
for i in range(len(tweets_data)):
    if i not in lst_error:
        lst = tweets_data[i]['created_at'].split(' ')
        date = (lst[5]+'.'+lst[1]+'.'+lst[2]+' '+lst[3])
        date_time = datetime.strptime(date, '%Y.%b.%d %H:%M:%S')
        tweets_data[i]['created_at'] =  date_time.replace(tzinfo=timezone.utc).astimezone(thai_tz).strptime(date, '%Y.%b.%d %H:%M:%S') # reset info and convert to thai timezone

In [ ]:
time = [tweets_data[i]['created_at'] for i in range(len(tweets_data)) if i not in lst_error]
language = [tweets_data[i]['lang'] for i in range(len(tweets_data)) if i not in lst_error]
source = [tweets_data[i]['source'][tweets_data[i]['source'].find('w">')+3:].replace('</a>', '') for i in range(len(tweets_data)) if i not in lst_error]

## Create Data frame

In [ ]:
df = pd.DataFrame({'time':time, 'language':language, 'source':source}) 

In [ ]:
# df['source'].value_counts()

## Add group column into data frame

In [ ]:
bins = list(range(0, 25*3600, 1*3600)) # Divide the time into 24 bin ( 1 hours / bin )
labels = ['0 pm','1 am','2 am','3 am','4 am',
          '5 am','6 am','7 am','8 am','9 am',
          '10 am','11 am','12 am','13 pm','14 pm',
         '15 pm','16 pm','17 pm','18 pm','19 pm',
          '20 pm','21 pm','22 pm','23 pm'] # set label
# print(list(range(0, 25, 2)))
# bins

In [ ]:
df['sec'] = df.time.dt.hour * 3600 + df.time.dt.minute*60 + df.time.dt.second # convert time unit to second.
df['group'] = pd.cut(df['sec'], bins=bins, labels=labels) # set bin label into group column.

In [ ]:
df.drop('sec', inplace=True, axis=1)

In [ ]:
# df.head()

## Hashtag to data frame

In [ ]:
hashtag_all = []
for j in range(len(tweets_data)):
    if j not in lst_error:
        lst = []
        for i in range(len(tweets_data[j]['entities']['hashtags'])):
            lst.append(tweets_data[j]['entities']['hashtags'][i]['text'])
        hashtag_all.append(lst)

In [ ]:
# hashtag_all

In [ ]:
df['hashtag'] = hashtag_all

In [ ]:
# df.head()

-----------

In [ ]:
# alltext = []

# try: # Check error in data
#     for i in range(len(tweets_data)):
#         if i not in lst_error: # ensure that message, not in the list of error.
#             if re.search('Microsoft PowerApps and Flow',tweets_data[i]['source']): # if tweet posted by Microsoft PowerApps, the key for access text will have only one then this condition must be checked first.
#                 text = tweets_data[i]['text']
#                 alltext.append(text)
#             elif re.search('^RT' ,tweets_data[i]['text']): # check type of tweet [RT = retweet]
#                 if re.search('…$' ,tweets_data[i]['text']): # check the message that full message or summary message.
#                     try: # in normal case
#                         text = tweets_data[i]['retweeted_status']['extended_tweet']['full_text'] # access text.
#                         alltext.append(text) # append to the alltext list.
# #                         print(text)
# #                         print('--------------------')
#                     except: # in case of a full message not in key 'extended_tweet'.
#                         text = tweets_data[i]['retweeted_status']['text'] # access text.
#                         alltext.append(text) # append to the alltext list.
# #                         print(text)
# #                         print('--------------------')
#                 else:
#                     if re.search('… https://' ,tweets_data[i]['text']): # check the text is condensed but the embedded link is unabridged.
#                         text = tweets_data[i]['extended_tweet']['full_text'] # access text.
#                         alltext.append(text) # append to the alltext list.
# #                       print(text)
# #                       print('--------------------')
#                     else:
#                         text = tweets_data[i]['retweeted_status']['text'] # access text.
#                         alltext.append(text) # append to the alltext list.
# #                     print(text)
# #                     print('--------------------')
#             else: # in case of normal tweets
#                 if re.search('… https//' ,tweets_data[i]['text']): # check the text is condensed but the embedded link is unabridged.
#                     text = tweets_data[i]['extended_tweet']['full_text'] # access text.
#                     alltext.append(text) # append to the alltext list.
# #                     print(text)
# #                     print('--------------------')
#                 else:
#                     text = tweets_data[i]['text'] # access text.
#                     alltext.append(text) # append to the alltext list.
# #                     print(text)
# #                     print('--------------------')
# except:
#     print(f'ERROR FOUND!! INDEX : {i}')

In [ ]:
# len(alltext)

In [ ]:
df.to_csv('prepared03.csv', encoding = 'utf8')

In [ ]:
# data = pd.read_csv('prepared01.csv')

In [ ]:
# data